# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [14]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os
import geoviews as gv
import geoviews.feature as gf
import xarray as xr
from cartopy import crs
# Import API key
from api_keys import geoapify_key

In [15]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,airai,-8.9266,125.4092,295.40,73,20,1.08,TL,1681442249
1,1,coruripe,-10.1256,-36.1756,297.13,75,81,2.31,BR,1681442708
2,2,cayenne,4.9333,-52.3333,297.17,94,75,1.03,GF,1681442082
3,3,victoria,22.2855,114.1577,300.18,82,100,4.47,HK,1681442187
4,4,chuy,-33.6971,-53.4616,285.56,71,4,3.11,UY,1681442254


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 0.65,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
not_holiday_conditions = []

# Drop any rows with null values
for x in range(len(city_data_df)):
    if city_data_df["Max Temp"][x] > 300 or city_data_df["Max Temp"][x] < 274 or city_data_df["Wind Speed"][x] > 4.5 or city_data_df["Cloudiness"][x] != 0:
        not_holiday_conditions.append(x)
vacation_df = city_data_df.drop(not_holiday_conditions)
vacation_df.dropna(inplace = True)
# Display sample data
vacation_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,hilo,19.7297,-155.0900,298.00,73,0,2.06,US,1681442841
16,16,mataura,-46.1927,168.8643,290.82,59,0,3.54,NZ,1681442004
19,19,new norfolk,-42.7826,147.0587,289.22,44,0,2.22,AU,1681442247
22,22,punta arenas,-53.1500,-70.9167,274.21,93,0,3.09,CL,1681442844
32,32,ushuaia,-54.8000,-68.3000,274.96,87,0,3.60,AR,1681442850


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_df.drop(columns = ["City_ID", "Max Temp", "Cloudiness", "Wind Speed", "Date"]).copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5, "Hotel Name", 0)

# Display sample data
hotel_df.head()

,City,Lat,Lng,Humidity,Country,Hotel Name
15,hilo,19.7297,-155.0900,73,US,0
16,mataura,-46.1927,168.8643,59,NZ,0
19,new norfolk,-42.7826,147.0587,44,AU,0
22,punta arenas,-53.1500,-70.9167,93,CL,0
32,ushuaia,-54.8000,-68.3000,87,AR,0


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": "1",
    "apiKey": f"{geoapify_key}"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, "Lat"]
    lon = hotel_df.loc[index, "Lng"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hilo - nearest hotel: Dolphin Bay Hotel
mataura - nearest hotel: No hotel found
new norfolk - nearest hotel: Woodbridge on the Derwent
punta arenas - nearest hotel: Hotel Lacolet
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
fereydun kenar - nearest hotel: هتل کیش مهر
sarkikaraagac - nearest hotel: Otel Korkusuz
jamestown - nearest hotel: DoubleTree Jamestown
east london - nearest hotel: No hotel found
coihaique - nearest hotel: No hotel found
suamico - nearest hotel: Country Inn & Suites
truth or consequences - nearest hotel: Travelodge
butte - nearest hotel: No hotel found
neuquen - nearest hotel: Hotel Argentina 307
russell - nearest hotel: No hotel found
coos bay - nearest hotel: No hotel found
stephenville - nearest hotel: No hotel found
gat - nearest hotel: No hotel found
birjand - nearest hotel: مهمانسرای جهانگردی بیرجند
nukus - nearest hotel: Qaraqalpaqstan Respublikası Ministrler Keńesi rezidenciyası
havelock - nearest hotel: Quality Inn Havelock 

,City,Lat,Lng,Humidity,Country,Hotel Name
15,hilo,19.7297,-155.0900,73,US,Dolphin Bay Hotel
16,mataura,-46.1927,168.8643,59,NZ,No hotel found
19,new norfolk,-42.7826,147.0587,44,AU,Woodbridge on the Derwent
22,punta arenas,-53.1500,-70.9167,93,CL,Hotel Lacolet
32,ushuaia,-54.8000,-68.3000,87,AR,Apart Hotel Aires del Beagle
38,fereydun kenar,36.6864,52.5225,80,IR,هتل کیش مهر
62,sarkikaraagac,38.0794,31.3664,72,TR,Otel Korkusuz
66,jamestown,42.0970,-79.2353,59,US,DoubleTree Jamestown
78,east london,-33.0153,27.9116,67,ZA,No hotel found
81,coihaique,-45.5752,-72.0662,70,CL,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 0.65,
    color = "City",
    hover_cols = "all"
)


# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,index,Country,Hotel Name)